In [2]:
import pickle
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level

set_log_level('ERROR')

In [3]:

loaded_model = pickle.load(open('../models/inflow_forecast_model_with_weather.pkl', 'rb'))


In [13]:

target_column = 'Total Inflow hrly'
df = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/since-2020.csv')
df.ds = pd.to_datetime(df.ds)
df = df.drop(['Date', 'Time'], axis=1)
df = df.sort_values(by='ds', ascending=True)
df.rename(columns = {target_column:'y', 'Adm. requests cum':"Adm requests cum", 'Pts.waiting for admission CUM':'Pts waiting for admission CUM'}, inplace = True)
df.dropna(inplace=True)
df = df.reset_index(drop=True)
regressors = df.columns.tolist()
regressors.remove('y')
regressors.remove('ds')


In [14]:
df

,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,y,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,GARAGE patient TBS,Consults > 2h in Vertical Except IM,Consult for IM >4h in Vertical,Plain films reqs > 2 hr in Vertical,CTs reqs > 2 hrs in Vertical,Psych Stretcher Pts1pt,Psych pts waiting for admission,ds,Total Pod TBS,Total Vertical TBS
0,9,108,5,142,14,250,6,39,0,27,...,1.0,1,0,0,3,0,0,2021-07-26 19:00:00,10,18.0
1,5,113,6,148,11,261,2,41,0,33,...,1.0,1,0,0,3,1,0,2021-07-26 20:00:00,12,9.0
2,5,118,2,150,7,268,1,42,0,34,...,0.0,1,0,0,4,4,0,2021-07-26 21:00:00,8,4.0
3,5,123,8,158,13,281,2,44,0,34,...,0.0,1,0,0,3,5,0,2021-07-26 22:00:00,9,6.0
4,3,126,7,165,10,291,3,47,0,37,...,1.0,0,0,1,3,6,0,2021-07-26 23:00:00,10,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8904,5,34,8,29,13,63,1,13,1,9,...,0.0,5,0,0,0,7,4,2022-08-04 10:00:00,5,11.0
8905,8,42,8,37,16,79,2,15,2,9,...,0.0,6,0,0,0,8,4,2022-08-04 11:00:00,3,18.0
8906,8,50,13,50,21,100,0,15,0,14,...,0.0,2,0,1,1,11,5,2022-08-04 12:00:00,1,27.0
8907,8,58,12,62,20,120,2,17,0,14,...,0.0,3,0,0,3,11,5,2022-08-04 13:00:00,3,27.0


In [15]:
regressors

['Stretcher Pts hrly',
 'Stretcher Pts cum',
 'Ambulatory Pts hrly',
 'Ambulatory Pts cum',
 'Total Inflow cum',
 'Ambulances hrly',
 'Ambulances cum',
 'FLS hrly',
 'Adm requests cum',
 'Admissions cum',
 'Pts waiting for admission CUM',
 'Total Stretcher pts',
 'Triage hallway pts',
 'Triage hallway pts TBS',
 'Re-Oriented Nurse cum',
 'Re-Oriented MD QTrack D/C',
 'Re-Oriented MD QTrack NotD/C',
 'Resus Pts',
 'Totalpts in PODs except Psych',
 'Green Pts',
 'Green Pts TBS',
 'Yellow PTS',
 'Yellow Pts TBS',
 'Orancge Pts except psych',
 'Orange Pts TBS',
 'Consults > 2h in PODS except IM',
 'Consult for IM >4h in PODS',
 'Plain films reqs > 2 h in PODs',
 'CTs reqs > 2 h in PODs',
 'Post POD (Family room)',
 'Stretcher Pts in Vertical',
 'Stretcher Pts TBS in Vertical',
 'Stretcher Pts in Vertical on Lazyboy',
 'Vertical Pts Waiting for Results',
 'Ambulatory Pts in Vertical',
 'Ambulatory Pts TBS in Vertical',
 'QTrack Patients TBS',
 'GARAGE patient TBS',
 'Consults > 2h in Vertic

In [16]:

weather = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/weatherArchiveAndForecast.csv')
weather.ds = pd.to_datetime(weather.ds)
weather = weather.sort_values(by='ds', ascending=True)
weather.tail()


,ds,temp,dew,humidity,precip,windspeed,sealevelpressure
22725,2022-08-05 19:00:00,25.9,13.3,45.69,0.0,10.4,1022.0
22726,2022-08-05 20:00:00,23.9,13.4,51.82,0.0,11.2,1022.0
22727,2022-08-05 21:00:00,21.8,13.4,58.86,0.0,6.1,1023.0
22728,2022-08-05 22:00:00,20.6,13.7,64.61,0.0,4.0,1023.0
22729,2022-08-05 23:00:00,19.6,14.1,70.54,0.0,5.8,1023.0


In [17]:

last_timestamp = df.loc[len(df)-1].ds
last_timestamp

Timestamp('2022-08-04 14:00:00')

In [18]:

df = df.merge(weather, on='ds')
df.tail()


,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,y,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,Psych pts waiting for admission,ds,Total Pod TBS,Total Vertical TBS,temp,dew,humidity,precip,windspeed,sealevelpressure
8891,5,34,8,29,13,63,1,13,1,9,...,4,2022-08-04 10:00:00,5,11.0,23.8,18.9,73.63,0.0,5.4,1013.4
8892,8,42,8,37,16,79,2,15,2,9,...,4,2022-08-04 11:00:00,3,18.0,24.2,19.5,74.60,0.0,7.6,1013.8
8893,8,50,13,50,21,100,0,15,0,14,...,5,2022-08-04 12:00:00,1,27.0,26.0,20.0,69.54,0.0,7.6,1014.0
8894,8,58,12,62,20,120,2,17,0,14,...,5,2022-08-04 13:00:00,3,27.0,25.4,20.4,73.44,0.0,5.4,1014.3
8895,6,64,13,75,19,139,1,18,1,18,...,6,2022-08-04 14:00:00,3,30.0,26.0,21.5,76.27,0.1,14.0,1013.0


In [22]:

df['ID']='test'
df.tail()


,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,y,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,ds,Total Pod TBS,Total Vertical TBS,temp,dew,humidity,precip,windspeed,sealevelpressure,ID
8891,5,34,8,29,13,63,1,13,1,9,...,2022-08-04 10:00:00,5,11.0,23.8,18.9,73.63,0.0,5.4,1013.4,test
8892,8,42,8,37,16,79,2,15,2,9,...,2022-08-04 11:00:00,3,18.0,24.2,19.5,74.60,0.0,7.6,1013.8,test
8893,8,50,13,50,21,100,0,15,0,14,...,2022-08-04 12:00:00,1,27.0,26.0,20.0,69.54,0.0,7.6,1014.0,test
8894,8,58,12,62,20,120,2,17,0,14,...,2022-08-04 13:00:00,3,27.0,25.4,20.4,73.44,0.0,5.4,1014.3,test
8895,6,64,13,75,19,139,1,18,1,18,...,2022-08-04 14:00:00,3,30.0,26.0,21.5,76.27,0.1,14.0,1013.0,test


In [24]:

future = pd.concat([df.tail(48),weather[weather.ds>last_timestamp].head(12)])
future['ID']='test'
future


,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,y,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,ds,Total Pod TBS,Total Vertical TBS,temp,dew,humidity,precip,windspeed,sealevelpressure,ID
8848,16.0,82.0,5.0,72.0,21.0,154.0,4.0,19.0,0.0,15.0,...,2022-08-02 13:00:00,3.0,29.0,26.9,19.0,61.66,0.00,5.4,1005.7,test
8849,8.0,90.0,8.0,80.0,16.0,170.0,4.0,23.0,0.0,16.0,...,2022-08-02 14:00:00,2.0,25.0,27.7,19.4,60.31,0.00,5.4,1006.1,test
8850,17.0,107.0,2.0,82.0,19.0,189.0,3.0,26.0,0.0,18.0,...,2022-08-02 15:00:00,3.0,28.0,26.9,18.0,57.94,0.00,5.5,1006.8,test
8851,11.0,117.0,8.0,91.0,19.0,208.0,1.0,27.0,1.0,20.0,...,2022-08-02 16:00:00,2.0,31.0,25.9,17.0,57.65,0.00,5.5,1007.3,test
8852,7.0,124.0,8.0,99.0,15.0,223.0,0.0,27.0,0.0,23.0,...,2022-08-02 17:00:00,4.0,25.0,25.4,14.4,50.28,0.00,11.2,1008.2,test
8853,12.0,136.0,9.0,108.0,21.0,244.0,5.0,32.0,0.0,27.0,...,2022-08-02 18:00:00,5.0,25.0,25.3,14.2,49.93,0.00,7.6,1008.9,test
8854,8.0,144.0,9.0,117.0,17.0,261.0,3.0,35.0,0.0,34.0,...,2022-08-02 19:00:00,9.0,29.0,23.9,14.0,53.58,0.00,5.4,1010.1,test
8855,4.0,148.0,0.0,117.0,4.0,265.0,1.0,36.0,0.0,36.0,...,2022-08-02 20:00:00,8.0,27.0,21.9,13.0,56.67,0.00,9.4,1011.1,test
8856,5.0,153.0,8.0,125.0,13.0,278.0,1.0,37.0,2.0,38.0,...,2022-08-02 21:00:00,6.0,30.0,20.8,11.9,56.41,0.00,9.4,1012.4,test
8857,6.0,159.0,8.0,133.0,14.0,292.0,2.0,39.0,0.0,41.0,...,2022-08-02 22:00:00,4.0,31.0,19.9,11.0,56.20,0.00,9.4,1013.1,test


In [25]:

# future = loaded_model.make_future_dataframe(df, periods=12) # periods=m.n_forecasts, n_historic_predictions=False

forecast = loaded_model.predict(future, raw=True, decompose=False)


In [26]:

start = forecast.values.tolist()[0][0]
forecast_length = len(forecast.values.tolist()[0][1:])


In [27]:

forecast_output = pd.DataFrame(columns=['ds','inflow','timestamp'])
forecast_output['ds'] = pd.date_range(start=start, periods=forecast_length, freq='H')
forecast_output['inflow'] = forecast.values.tolist()[0][1:]
forecast_output['timestamp'] = pd.Timestamp.now().round('S').replace(tzinfo=None)
forecast_output.to_csv('../forecasts/inflow_with_weather.csv', index=False)

In [28]:
forecast_output

,ds,inflow,timestamp
0,2022-08-04 13:00:00,15.695422,2022-08-04 14:27:04
1,2022-08-04 14:00:00,14.859688,2022-08-04 14:27:04
2,2022-08-04 15:00:00,18.869854,2022-08-04 14:27:04
3,2022-08-04 16:00:00,17.698275,2022-08-04 14:27:04
4,2022-08-04 17:00:00,15.793342,2022-08-04 14:27:04
5,2022-08-04 18:00:00,14.269565,2022-08-04 14:27:04
6,2022-08-04 19:00:00,12.983377,2022-08-04 14:27:04
7,2022-08-04 20:00:00,11.200150,2022-08-04 14:27:04
8,2022-08-04 21:00:00,11.938907,2022-08-04 14:27:04
9,2022-08-04 22:00:00,10.661190,2022-08-04 14:27:04
